In [1]:
# импортируем необходимые библиотеки и классы
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from classic_stacking import ClassicStacking

In [2]:
# загружаем исторический набор и набор новых данных
train = pd.read_csv('Data/ottogroup_train.csv')
test = pd.read_csv('Data/ottogroup_test.csv')

In [3]:
# формируем массив меток и массив признаков
labels = train.pop('target')

In [4]:
# сохраняем id набора новых данных
test_id = test['id']

# удаляем id из обоих наборов
train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)

In [5]:
# создаем экземпляр класса XGBClassifier
# (модель должна быть уже настроена) -
# первый базовый алгоритм
xgbst_model = XGBClassifier(learning_rate=0.06,
                                objective='multi:softprob',
                                subsample=0.8,
                                gamma=1,
                                colsample_bytree=0.8,
                                colsample_bylevel=0.8,
                                min_child_weight=0.8,
                                random_state=42,
                                max_depth=9,
                                n_estimators=800)

# создаем экземпляр класса CatBoostClassifier
# (модель должна быть уже настроена) -
# второй базовый алгоритм
ctbst_model = CatBoostClassifier(learning_rate=0.035,
                                     loss_function='MultiClass',
                                     depth=11,
                                     bagging_temperature=0.2,
                                     rsm=0.1,
                                     iterations=2500,
                                     random_strength=0,
                                     random_seed=42,
                                     logging_level='Silent')

# создаем экземпляр класса LogisticRegression
# (модель должна быть уже настроена) -
# третий базовый алгоритм
logreg_model = LogisticRegression(solver='liblinear')

# создаем конвейер для логистической регрессии
logreg_pipe = Pipeline([('scaler', StandardScaler()), ('logreg', logreg_model)])

In [6]:
# создаем экземпляр класса ClassicStacking
s = ClassicStacking(estimators=[xgbst_model, ctbst_model, logreg_pipe],
                    final_estimator=ctbst_model, cv=5, 
                    stack_method='predict_proba', passthrough=True)
# обучаем базовые алгоритмы и мета-алгоритм
s.fit(train, labels)
# получаем вероятности
final_proba = s.predict(train, labels, test)

In [7]:
# извлекаем метки классов
classes = np.unique(labels)
# формируем посылку
submission = pd.DataFrame(data=final_proba, columns=classes)
submission.insert(0, 'id', test_id)
submission.to_csv('own_cv_stacking_on_cl_probs_with_concat.csv', index=False)